In [1]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
data_backup = pd.read_csv('../deliveries.csv')
data = data_backup.copy()
data.columns

Index(['match_id', 'season', 'start_date', 'venue', 'innings', 'ball',
       'batting_team', 'bowling_team', 'striker', 'non_striker', 'bowler',
       'runs_off_bat', 'extras', 'wides', 'noballs', 'byes', 'legbyes',
       'penalty', 'wicket_type', 'player_dismissed', 'other_wicket_type',
       'other_player_dismissed'],
      dtype='object')

In [3]:
prev_data = pd.read_csv('../previous_data_per.csv')
prev_data['%'] = prev_data['Won'] / prev_data['Mat']
prev_data['%'] = prev_data['%'].apply(lambda x: round(x, 2))

In [4]:
original_df = data
# Convert 'start_date' to datetime if it's not already in datetime format
original_df['start_date'] = pd.to_datetime(original_df['start_date'])

# Group by match_id, innings, and venue
grouped_df = original_df.groupby(['match_id', 'innings', 'venue'])

# Calculate the required statistics for each group
result_df = grouped_df.agg({
    'batting_team': 'first',
    'bowling_team': 'first',
    'wides': 'sum',
    'noballs': 'sum',
    'byes': 'sum',
    'legbyes': 'sum',
    'runs_off_bat': 'sum',
    'ball': 'max',
    'wicket_type': lambda x: x.notnull().sum(),
    'start_date': 'first'
}).reset_index()

# Rename columns
result_df.columns = ['match_id', 'innings', 'venue', 'batting_team', 'bowling_team',
                      'total_wides', 'total_noballs', 'total_byes', 'total_legbyes', 'total_runs_per_innings_match',
                       'last_ball', 'total_wickets', 'date']

# Calculate 'total_overs_played' based on the 'ball' column
result_df['total_overs_played'] = result_df['last_ball'].apply(lambda x: min((int(x) + round((x % 1) * 10, 4) / 6), 50.0))

# Change dtype of 'total_wides' and 'total_noballs' to int
result_df[['total_wides', 'total_noballs', 'total_byes', 'total_legbyes']] = result_df[['total_wides', 'total_noballs', 'total_byes', 'total_legbyes']].astype(int)

# Add 'total_wides' and 'total_noballs' to 'total_runs_per_innings_match' per innings
result_df['total_runs_per_innings_match'] += result_df['total_wides'] + result_df['total_noballs'] + result_df['total_byes'] + result_df['total_legbyes']

result_df.drop(['last_ball'], axis=1, inplace=True)

In [5]:
result_df.head()

,match_id,innings,venue,batting_team,bowling_team,total_wides,total_noballs,total_byes,total_legbyes,total_runs_per_innings_match,total_wickets,date,total_overs_played
0,1,1,"Narendra Modi Stadium, Ahmedabad",England,New Zealand,6,0,0,0,282,9,2023-10-05,50.000000
1,1,2,"Narendra Modi Stadium, Ahmedabad",New Zealand,England,3,0,4,1,283,1,2023-10-05,36.333333
2,2,1,"Rajiv Gandhi International Stadium, Uppal, Hyd...",Pakistan,Netherlands,8,1,0,0,286,10,2023-10-06,49.000000
3,2,2,"Rajiv Gandhi International Stadium, Uppal, Hyd...",Netherlands,Pakistan,9,0,0,0,205,10,2023-10-06,41.166667
4,3,1,"Himachal Pradesh Cricket Association Stadium, ...",Afghanistan,Bangladesh,8,0,0,0,156,10,2023-10-07,37.333333


In [6]:
result_df.drop(['total_wickets','date','match_id','total_wides', 'total_noballs', 'total_byes', 'total_legbyes'], axis=1, inplace=True)
result_df.head()

,innings,venue,batting_team,bowling_team,total_runs_per_innings_match,total_overs_played
0,1,"Narendra Modi Stadium, Ahmedabad",England,New Zealand,282,50.000000
1,2,"Narendra Modi Stadium, Ahmedabad",New Zealand,England,283,36.333333
2,1,"Rajiv Gandhi International Stadium, Uppal, Hyd...",Pakistan,Netherlands,286,49.000000
3,2,"Rajiv Gandhi International Stadium, Uppal, Hyd...",Netherlands,Pakistan,205,41.166667
4,1,"Himachal Pradesh Cricket Association Stadium, ...",Afghanistan,Bangladesh,156,37.333333


In [7]:
final_df = pd.merge(result_df,prev_data,left_on=['batting_team','bowling_team'],right_on=['Team','Opposition'],how='left')
final_df.drop(['Team','Opposition','Mat','Won','Lost'],axis=1,inplace=True)
final_df['%'].fillna(0.50,inplace=True)
final_df.head()

,innings,venue,batting_team,bowling_team,total_runs_per_innings_match,total_overs_played,%
0,1,"Narendra Modi Stadium, Ahmedabad",England,New Zealand,282,50.000000,0.46
1,2,"Narendra Modi Stadium, Ahmedabad",New Zealand,England,283,36.333333,0.47
2,1,"Rajiv Gandhi International Stadium, Uppal, Hyd...",Pakistan,Netherlands,286,49.000000,0.50
3,2,"Rajiv Gandhi International Stadium, Uppal, Hyd...",Netherlands,Pakistan,205,41.166667,0.50
4,1,"Himachal Pradesh Cricket Association Stadium, ...",Afghanistan,Bangladesh,156,37.333333,0.38


In [8]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

mapping = {}
categorical_columns = ['venue','batting_team','bowling_team']

for column in categorical_columns:
    final_df[column] = le.fit_transform(final_df[column])
    mapping[column] = dict(zip(le.classes_, le.transform(le.classes_)))

In [9]:
X=final_df.drop('total_overs_played',axis='columns')
y=final_df['total_overs_played']

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)

size_scaler = preprocessing.StandardScaler().fit(X_train)
X_train_scaled = size_scaler.transform(X_train)
X_test_scaled = size_scaler.transform(X_test)
X_train_scaled.shape, X_test_scaled.shape

((51, 6), (13, 6))

In [11]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_train,y_train)
model.score(X_test,y_test)

0.6608080146921232

In [12]:
y_pred = model.predict(X_test)
y_pred

array([54.46758934, 44.25522787, 48.2253875 , 53.09561074, 36.23371855,
       44.7783668 , 50.0020989 , 52.46595684, 39.25257846, 36.52107479,
       43.98276931, 39.64346789, 46.89290285])

In [13]:
y_test

52    49.333333
58    49.666667
0     50.000000
44    50.000000
5     34.666667
36    49.833333
16    50.000000
12    50.000000
25    40.500000
61    32.500000
56    50.000000
9     41.333333
40    50.000000
Name: total_overs_played, dtype: float64

In [14]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler


model = keras.Sequential([
    layers.Input(shape=(X_train.shape[1],)),  # Input layer
    layers.Dense(32, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(8, activation='relu'),
    layers.Dense(1, activation='linear')  # Output layer with 1 unit and linear activation
])

model.compile(optimizer='adam', loss='mse', metrics=['mae'])

early_stopping = EarlyStopping(monitor='val_loss', patience=10)

model.fit(X_train, y_train, epochs=500, batch_size=1, verbose=1, callbacks=[early_stopping], validation_split=0.2)

Epoch 1/500
40/40 [==============================] - 1s 8ms/step - loss: 212.0442 - mae: 12.1655 - val_loss: 66.1185 - val_mae: 6.6103
Epoch 2/500
40/40 [==============================] - 0s 4ms/step - loss: 89.8422 - mae: 7.9772 - val_loss: 58.4612 - val_mae: 6.4584
Epoch 3/500
40/40 [==============================] - 0s 5ms/step - loss: 92.2469 - mae: 8.2839 - val_loss: 67.3165 - val_mae: 6.5859
Epoch 4/500
40/40 [==============================] - 0s 5ms/step - loss: 93.5773 - mae: 7.8282 - val_loss: 59.3064 - val_mae: 6.4008
Epoch 5/500
40/40 [==============================] - 0s 4ms/step - loss: 95.5056 - mae: 7.9599 - val_loss: 57.0095 - val_mae: 6.4155
Epoch 6/500
40/40 [==============================] - 0s 4ms/step - loss: 100.0758 - mae: 8.4003 - val_loss: 60.9487 - val_mae: 6.4161
Epoch 7/500
40/40 [==============================] - 0s 4ms/step - loss: 102.9171 - mae: 8.4491 - val_loss: 69.7260 - val_mae: 6.5387
Epoch 8/500
40/40 [==============================] - 0s 4ms/step 

In [15]:
train_loss, train_mae = model.evaluate(X_train, y_train, verbose=1)
print(f"Training MAE: {train_mae}")

# Evaluate on the test set
test_loss, test_mae = model.evaluate(X_test, y_test, verbose=1)
print(f"Test MAE: {test_mae}")

2/2 [==============================] - 0s 3ms/step - loss: 63.2827 - mae: 6.6948
Training MAE: 6.694809913635254
1/1 [==============================] - 0s 27ms/step - loss: 87.9814 - mae: 8.2228
Test MAE: 8.222846031188965


In [24]:
import pickle as pkl

In [25]:
pkl.dump(model, open('../pickle/over_predict_model', 'wb'))